## Hash function

## HMAC

### HMAC-SHA1

In [1]:
import hmac
import hashlib

# 비밀 키와 메시지 설정
secret_key = b"super_secret_key"
message = b"Hello, HMAC-SHA1!"

# HMAC-SHA1 생성
hmac_sha1 = hmac.new(secret_key, message, hashlib.sha1)

# 출력 (16진수 표현)
print("HMAC-SHA1:", hmac_sha1.hexdigest())


HMAC-SHA1: d5fa12b8d0ff0cf875426c08232ae790a9c8101e


In [15]:
import hashlib

# 비밀 키와 메시지 설정
secret_key = b"super_secret_key"
message = b"Hello, HMAC-SHA1!"

# HMAC에서 사용하는 블록 크기
block_size = 64  # SHA-1, SHA-256은 64바이트 블록 사용

def padding_bytes(data):
    if len(data) > block_size:
        data = hashlib.sha256(data).digest()  # 키가 크면 해싱
    data = data.ljust(block_size, b'\x00')  # 짧으면 0으로 패딩
    return data     

def xor_bytes(a, b):
    return bytes([x ^ y for x, y in zip(a, b)])

def hmac_function(key, message, hashfunc):
    # padding 
    key = padding_bytes(key)
    # opad, ipad 생성 
    opad = bytes([0x5c] * 64)
    ipad = bytes([0x36] * 64)
    # xor 1차 
    res1 = xor_bytes(key,opad)
    res2 = xor_bytes(key,ipad)
    res3 = res2 + message 
    res4 = hashfunc(res3).digest()
    res5 = res1 + res4
    res6 = hashfunc(res5)
    print(f"""
        key   : {key.hex()}
        res 1 : {res1.hex()}
        res 2 : {res2.hex()}
        res 3 : {res3.hex()}
        res 4 : {res4.hex()}
        res 5 : {res5.hex()}
        res 6 : {res6.hexdigest()}
    """)
    return res6 

hmac_sha1 = hmac_function(secret_key, message, hashlib.sha1)
print("HMAC-SHA1:", hmac_sha1.hexdigest())



        key   : 73757065725f7365637265745f6b6579000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
        res 1 : 2f292c392e032f393f2e3928033739255c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c
        res 2 : 454346534469455355445342695d534f363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636
        res 3 : 454346534469455355445342695d534f36363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363648656c6c6f2c20484d41432d5348413121
        res 4 : 52eff803a9963147e3ca4b7e3498bc7133771eee
        res 5 : 2f292c392e032f393f2e3928033739255c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c52eff803a9963147e3ca4b7e3498bc7133771eee
        res 6 : d5fa12b8d0ff0cf875426c08232ae790a9c8101e
    
HMAC-SHA1: d5fa12b8d0ff0cf875426c08232ae790a9c8101e


# SHA1

In [ ]:
import struct

def leftrotate(n, b):
    """32비트 정수 n을 b 비트만큼 왼쪽 순환 이동합니다."""
    return ((n << b) | (n >> (32 - b))) & 0xffffffff

def sha1(data):
    # 데이터가 바이트열이 아니면 인코딩
    if isinstance(data, str):
        data = data.encode()

    # 초기 해시값 (SHA-1 표준)
    h0 = 0x67452301
    h1 = 0xEFCDAB89
    h2 = 0x98BADCFE
    h3 = 0x10325476
    h4 = 0xC3D2E1F0

    # 전처리: 패딩
    original_byte_len = len(data)
    original_bit_len = original_byte_len * 8

    # 1. 0x80 바이트 추가
    data += b'\x80'

    # 2. 데이터를 64바이트의 배수가 될 때까지 0x00 추가
    while (len(data) % 64) != 56:
        data += b'\x00'

    # 3. 원래 메시지 길이(비트 단위)를 64비트 빅엔디언 형식으로 추가
    data += struct.pack('>Q', original_bit_len)

    # 데이터를 64바이트 블록으로 처리
    for i in range(0, len(data), 64):
        block = data[i:i+64]
        # 16개의 32비트 워드로 블록 분할 (빅엔디언)
        w = list(struct.unpack('>16I', block))
        # 워드 확장: 16번째부터 79번째까지
        for j in range(16, 80):
            w.append(leftrotate(w[j-3] ^ w[j-8] ^ w[j-14] ^ w[j-16], 1))

        # 초기 값 할당
        a = h0
        b = h1
        c = h2
        d = h3
        e = h4

        # 80라운드 반복
        for j in range(80):
            if 0 <= j <= 19:
                f = (b & c) | ((~b) & d)
                k = 0x5A827999
            elif 20 <= j <= 39:
                f = b ^ c ^ d
                k = 0x6ED9EBA1
            elif 40 <= j <= 59:
                f = (b & c) | (b & d) | (c & d)
                k = 0x8F1BBCDC
            else:  # 60 <= j <= 79
                f = b ^ c ^ d
                k = 0xCA62C1D6

            temp = (leftrotate(a, 5) + f + e + k + w[j]) & 0xffffffff
            e = d
            d = c
            c = leftrotate(b, 30)
            b = a
            a = temp

        # 내부 해시값 업데이트
        h0 = (h0 + a) & 0xffffffff
        h1 = (h1 + b) & 0xffffffff
        h2 = (h2 + c) & 0xffffffff
        h3 = (h3 + d) & 0xffffffff
        h4 = (h4 + e) & 0xffffffff

    # 최종 해시값을 40자리 16진수 문자열로 생성
    return '{:08x}{:08x}{:08x}{:08x}{:08x}'.format(h0, h1, h2, h3, h4)


## SHA-256

In [14]:
import struct

# SHA-256에서 사용되는 상수 (K 값)
K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2,
]

# 초기 해시 값 (SHA-256)
H = [
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19,
]

# 비트 연산 함수
def right_rotate(value, shift, size=32):
    """ 오른쪽 순환 시프트 연산 """
    return (value >> shift) | (value << (size - shift)) & 0xFFFFFFFF

# SHA-256 패딩 함수
def pad_message(message):
    """ 메시지를 512비트(64바이트) 배수로 패딩 """
    message_length = len(message)
    bit_length = message_length * 8

    # 1비트를 추가하고 나머지는 0으로 채움
    message += b'\x80'
    while (len(message) + 8) % 64 != 0:
        message += b'\x00'

    # 원래 메시지 길이를 64비트(big-endian)로 추가
    message += struct.pack(">Q", bit_length)
    return message

# SHA-256 해시 함수
def sha256(data):
    """ SHA-256 직접 구현 """
    # 패딩 적용
    padded_data = pad_message(data)

    # 512비트 블록 단위로 처리
    for i in range(0, len(padded_data), 64):
        chunk = padded_data[i:i+64]

        # 16개의 32비트 워드 (W) 생성
        W = list(struct.unpack(">16L", chunk)) + [0] * 48

        # 메시지 스케줄링 확장
        for j in range(16, 64):
            s0 = right_rotate(W[j-15], 7) ^ right_rotate(W[j-15], 18) ^ (W[j-15] >> 3)
            s1 = right_rotate(W[j-2], 17) ^ right_rotate(W[j-2], 19) ^ (W[j-2] >> 10)
            W[j] = (W[j-16] + s0 + W[j-7] + s1) & 0xFFFFFFFF

        # 초기 해시 값을 a~h로 설정
        a, b, c, d, e, f, g, h = H

        # 64 라운드 압축 연산 수행
        for j in range(64):
            S1 = right_rotate(e, 6) ^ right_rotate(e, 11) ^ right_rotate(e, 25)
            ch = (e & f) ^ ((~e) & g)
            temp1 = (h + S1 + ch + K[j] + W[j]) & 0xFFFFFFFF
            S0 = right_rotate(a, 2) ^ right_rotate(a, 13) ^ right_rotate(a, 22)
            maj = (a & b) ^ (a & c) ^ (b & c)
            temp2 = (S0 + maj) & 0xFFFFFFFF

            # 상태 업데이트
            h = g
            g = f
            f = e
            e = (d + temp1) & 0xFFFFFFFF
            d = c
            c = b
            b = a
            a = (temp1 + temp2) & 0xFFFFFFFF

        # 최종 해시 값 업데이트
        H[0] = (H[0] + a) & 0xFFFFFFFF
        H[1] = (H[1] + b) & 0xFFFFFFFF
        H[2] = (H[2] + c) & 0xFFFFFFFF
        H[3] = (H[3] + d) & 0xFFFFFFFF
        H[4] = (H[4] + e) & 0xFFFFFFFF
        H[5] = (H[5] + f) & 0xFFFFFFFF
        H[6] = (H[6] + g) & 0xFFFFFFFF
        H[7] = (H[7] + h) & 0xFFFFFFFF

    # 최종 결과 (32바이트 해시 값)
    return b''.join(struct.pack(">I", h) for h in H)

# 테스트
key = b"super_secret_key"
hash_result = sha256(key)
print("SHA-256 (hex):", hash_result.hex())

SHA-256 (hex): 7f5aa1e2e4cce1e41cf9b93db36f87c82a790cedb958bcd7f711c305d21e8db2


# 머클트리 (Merkle Tree)

In [2]:
import hashlib

def sha256(data):
    return hashlib.sha256(data.encode('utf-8')).hexdigest()

def double_sha256(data):
    return hashlib.sha256(hashlib.sha256(data.encode('utf-8')).digest()).hexdigest()

def combine_hash_funcion(left, right):
    combined = left + right
    new_hash = double_sha256(combined)    
    return new_hash 

def merkle_tree(transactions):
    # Step 1: 트랜잭션을 SHA-256 해시로 변환
    current_layer = [double_sha256(tx) for tx in transactions]

    # Step 2: 머클 트리 생성
    while len(current_layer) > 1:
        next_layer = []

        # 홀수 개일 경우 마지막 항목을 복제
        if len(current_layer) % 2 == 1:
            current_layer.append(current_layer[-1])

        # 인접 두 개씩 묶어 해시
        for i in range(0, len(current_layer), 2):
            left = current_layer[i]
            right = current_layer[i+1]
            new_hash = combine_hash_funcion(left, right)
            next_layer.append(new_hash)

        current_layer = next_layer

    return current_layer[0]  # 머클 루트 반환


# 🧪 예시 트랜잭션들
transactions = [
    "tx1: Alice -> Bob 0.5 BTC",
    "tx2: Bob -> Charlie 0.2 BTC",
    "tx3: Charlie -> Dave 0.1 BTC",
    "tx4: Dave -> Eve 0.05 BTC", 
    "tx5: Eve -> Frank 0.01 BTC",
    "tx6: Frank -> Grace 0.005 BTC",
    "tx7: Grace -> Heidi 0.001 BTC", 
    "tx8: Heidi -> Alice 0.0005 BTC",
]



# Merkle Root 계산
merkle_root = merkle_tree(transactions[:8])

print("Merkle Root:", merkle_root)

Merkle Root: 046ab6924ded7b0df1eb3e03be0111e5d9c7dfe1458d19731fa1de95b0ad0888


In [3]:
# 🧪 예시 👻 위조 트랜잭션들
transactions = [
    "tx1: Alice -> Bob 0.5 BTC",
    "tx2: Bob -> Charlie 0.2 BTC",
    "tx3: Charlie -> Dave 0.1 BTC",
    "tx4: Dave -> Eve 0.05 BTC", 
    "tx5: Eve -> Frank 0.01 BTC",
    "tx6: Frank -> Grace 0.005 BTC",
    "tx7: Grace -> Alice 0.001 BTC", 
    "tx8: Heidi -> Alice 0.0005 BTC",
]

# Merkle Root 계산
merkle_root = merkle_tree(transactions[:8])

print("Merkle Root:", merkle_root)

Merkle Root: f7402c8fc7c82edf539c2d30d7a4a07f7c5e3ec4beafc164c6af7774230d8b77
